## Compare results for multiple results

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [19]:
%matplotlib widget

In [20]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [21]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [22]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    val_dr=dict_val['hist_val'].copy()
    val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins
    
    sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2
    
    size=len(dict_val['hist_val'])
    l1,l2=int(size*0.3),int(size*0.7)
    keys=['chi_1a','chi_1b','chi_1c','chi_1']
    
    for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
        chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})
    
    idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
#     chisqr_dict.update({'chi_sqr1':})
    
    chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
    chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data
    
    idx=60
    spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})
    
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_err'][:idx]**2)})
    
    return chisqr_dict

In [23]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
# s_val=np.load(fname,mmap_mode='r')[10000:20000][:,0,:,:]
s_val=np.load(fname,mmap_mode='r')[:8000][:,0,:,:]

print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(8000, 128, 128)


In [24]:
df_runs=pd.DataFrame([])

In [25]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:500][:,:,:,0])
dict1['num_img']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


In [26]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['run5_fixed_cosmology','run6_fixed_cosmology','run7_no_truncated_normal','run8_no_truncated_normal']
prefix='keras_'

for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key=prefix+strg
    fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:500]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


In [28]:

parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201029_164804_bsize64_with_spec'
lst=[(19, 68728), (14, 51341), (14, 52211), (11, 40132)]

prefix='lbann_train_1_'
for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=parent_dir+'/dump_outs/trainer0/model0/'+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_img']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)

{'label': 'lbann_train_1_19-68728', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201029_164804_bsize64_with_spec/dump_outs/trainer0/model0/sgd.training.epoch.19.step.68728_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_1_14-51341', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201029_164804_bsize64_with_spec/dump_outs/trainer0/model0/sgd.training.epoch.14.step.51341_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_1_14-52211', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201029_164804_bsize64_with_spec/dump_outs/trainer0/model0/sgd.training.epoch.14.step.52211_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_1_11-40132', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201029_164804_bsize64_with_spec/dump_outs/trainer0/model0/sgd.training.epoch.11.step.40132_gen_img_instance1_activati

In [12]:
# df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'fname', 'hist_bin_centers', 'hist_err',
       'hist_val', 'label', 'num_img', 'spec_err', 'spec_val'],
      dtype='object')

In [29]:
col_list=['label','fname', 'hist_bin_centers', 'hist_err', 'hist_val','spec_err', 'spec_val','chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2', 'num_img']
df_runs=df_runs.reindex(columns=col_list)


In [30]:
df_runs

,label,fname,hist_bin_centers,hist_err,hist_val,spec_err,spec_val,chi_1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_img
0,raw,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0028445197484184732, 0.0020816260811514454,...","[1.121347563774932, 0.9576931243390049, 0.2568...","[1987.871939779571, 1009.5002022488515, 686.43...","[87592.048827034, 62024.131669237926, 44898.48...",0.008450,0.000121,0.000127,0.008202,0.000028,4.089795,0.001934,63.929411,500.0
1,keras_0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003081765924486072, 0.0023479649643570986, ...","[1.1146491994998429, 0.9643402305198981, 0.260...","[1913.65866089898, 963.0815281971833, 644.6720...","[85906.63125591779, 60940.73459686716, 43971.7...",0.001476,0.000066,0.000471,0.000939,0.000039,4.081442,0.045119,2128.730078,500.0
2,keras_1,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0031456442853499706, 0.002349065715521853, ...","[1.1148820753244182, 0.9669777732485703, 0.261...","[2012.174777339885, 1015.3485395616655, 727.33...","[86439.30616832257, 60426.54691713405, 45079.8...",0.001721,0.000433,0.000196,0.001093,0.000091,4.180616,0.067723,2914.557541,500.0
3,keras_2,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0032299157726108983, 0.00241667307354839, 0...","[1.1439634352627102, 0.9270007224057393, 0.253...","[1877.6099696345757, 1000.744571148909, 636.93...","[82519.15650125599, 59058.62442671653, 43854.1...",0.003428,0.002287,0.000471,0.000670,0.001813,4.139944,0.017434,457.575206,500.0
4,keras_3,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0029680794378720322, 0.002208163391797882, ...","[1.0973298711470636, 0.9833819747814072, 0.269...","[2032.9510454630201, 978.7139317701136, 649.82...","[88968.60526771784, 62091.84879396162, 45370.3...",0.003392,0.001494,0.000660,0.001238,0.001118,4.010530,0.045451,2296.231956,500.0
5,lbann_train_1_19-68728,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0036411451715963644, 0.004359788966404566, ...","[1.1042957632415245, 0.9695437267456408, 0.270...","[6499.359485792675, 2933.2994540577047, 1939.9...","[98546.242787987, 64755.84196794778, 46416.146...",0.004025,0.001128,0.001403,0.001494,0.000467,8.392049,0.747184,7505.467287,64.0
6,lbann_train_1_14-51341,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0035905144479249705, 0.00413346274974495, 0...","[1.1015845863622782, 0.9823692456119091, 0.264...","[4644.547566596898, 1851.3605879061079, 1299.0...","[83265.12758159637, 58985.944679152955, 42540....",0.004046,0.001359,0.000941,0.001746,0.000841,7.705099,0.484557,5529.070892,64.0
7,lbann_train_1_14-52211,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.004492031790948151, 0.004485892622631931, 0...","[1.0001201059388973, 1.0887263434014742, 0.308...","[4036.100950540113, 2338.362276896788, 1362.53...","[90645.39607492834, 64379.198783835775, 44240....",0.060365,0.046971,0.003371,0.010023,0.033761,8.420582,0.095444,936.712039,64.0
8,lbann_train_1_11-40132,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003847950319336979, 0.004208181709523121, 0...","[1.0371434417296603, 1.0516394392954258, 0.291...","[4490.208590119496, 2303.9625419510494, 1514.6...","[82965.96009929478, 56920.57671969569, 41676.8...",0.026613,0.021301,0.001789,0.003523,0.016320,7.909093,0.095935,721.471710,64.0


### Compare chi-squares

In [31]:
chi_keys=['label','chi_1','chi_spec1', 'num_img', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','fname']
df_runs[chi_keys]

,label,chi_1,chi_spec1,num_img,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,fname
0,raw,0.008450,0.001934,500.0,0.000121,0.000127,0.008202,0.000028,4.089795,63.929411,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_0,0.001476,0.045119,500.0,0.000066,0.000471,0.000939,0.000039,4.081442,2128.730078,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_1,0.001721,0.067723,500.0,0.000433,0.000196,0.001093,0.000091,4.180616,2914.557541,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
3,keras_2,0.003428,0.017434,500.0,0.002287,0.000471,0.000670,0.001813,4.139944,457.575206,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
4,keras_3,0.003392,0.045451,500.0,0.001494,0.000660,0.001238,0.001118,4.010530,2296.231956,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,lbann_train_1_19-68728,0.004025,0.747184,64.0,0.001128,0.001403,0.001494,0.000467,8.392049,7505.467287,/global/cscratch1/sd/vpa/proj/cosmogan/results...
6,lbann_train_1_14-51341,0.004046,0.484557,64.0,0.001359,0.000941,0.001746,0.000841,7.705099,5529.070892,/global/cscratch1/sd/vpa/proj/cosmogan/results...
7,lbann_train_1_14-52211,0.060365,0.095444,64.0,0.046971,0.003371,0.010023,0.033761,8.420582,936.712039,/global/cscratch1/sd/vpa/proj/cosmogan/results...
8,lbann_train_1_11-40132,0.026613,0.095935,64.0,0.021301,0.001789,0.003523,0.016320,7.909093,721.471710,/global/cscratch1/sd/vpa/proj/cosmogan/results...


In [32]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec'],"Invalid mode %s"%(plot_type)

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    
#     plt.xscale('symlog',linthreshx=30)

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')

In [33]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec']))

interactive(children=(SelectMultiple(description='labels_list', options=('raw', 'keras_0', 'keras_1', 'keras_2…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [18]:
col_list=['label','chi_1','chi_spec1','num_img','chi_2','chi_spec2']
df_runs[col_list].head(11)

,label,chi_1,chi_spec1,num_img,chi_2,chi_spec2
0,raw,0.008450,0.001933,500.0,0.000028,64.768287
1,keras_0,0.001476,0.043014,500.0,0.000039,2012.300668
2,keras_1,0.001721,0.066744,500.0,0.000091,2852.620989
3,keras_2,0.003428,0.017024,500.0,0.001813,436.852882
4,keras_3,0.003392,0.044437,500.0,0.001118,2270.140782
5,pytorch_1_4-6900,0.083922,0.337696,128.0,0.000396,2766.129855
6,pytorch_1_4-6950,0.061624,0.270516,128.0,0.001870,2392.392696
7,pytorch_1_6-9600,0.184034,0.262732,128.0,0.020805,1833.085743
8,pytorch_1_5-8750,0.194877,0.166826,128.0,0.015194,1590.675360
9,pytorch_1_4-7200,3.715080,0.582573,500.0,0.008518,14621.148897


In [61]:
f_transform(160)

0.9512195120761451